In [2]:
import joblib
from requests import get
import requests
from bs4 import BeautifulSoup
import re
import time
import polars as pl
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier

In [3]:
#1 Avoir la dernière version

def get_latest_version():
    try:
        return requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    except:
        return "16.1.1"

def clean_html(raw_html):
    if not raw_html: return ""
    cleanr = re.compile('<.*?>')
    return re.sub(cleanr, ' ', raw_html).replace('\n', ' ').strip().lower()

def extract_stat_from_text(regex, text):
    match = re.search(regex, text)
    return float(match.group(1)) if match else 0.0

#2 Récupération des données complètes des objets via l'API officielle
def get_items_complete():
    version = get_latest_version()
    print(f"Patch : {version} - Téléchargement via API Officielle...")

    url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/item.json"
    response = requests.get(url)
    
    if response.status_code != 200:
        print("Erreur API Riot")
        return None

    data = response.json()['data']
    items_list = []

    print(f"Analyse de {len(data)} objets...")

    for item_id, details in data.items():
        # les bons objets seulement
        if details.get('maps', {}).get('11') is not True:
            continue

        gold = details.get('gold', {})
        if not gold.get('purchasable', False):
            continue

        if 'requiredChampion' in details:
            continue
            
        is_trinket = "Trinket" in details.get('tags', [])
        if gold['total'] == 0 and not is_trinket:
            continue

        #les éléments qeu je veux seulement
        stats = details.get('stats', {})
        
     
        tags = details.get('tags', []) 
        
        raw_desc = details.get('description', '')
        clean_desc = clean_html(raw_desc)
        
        row = {
            "ID": item_id,
            "Name": details['name'],
            "Price": gold['total'],
            "Sell_Price": gold['sell'],
            
            
            "AD": stats.get('FlatPhysicalDamageMod', 0),
            "AP": stats.get('FlatMagicDamageMod', 0),
            "HP": stats.get('FlatHPPoolMod', 0),
            "Armor": stats.get('FlatArmorMod', 0),
            "MR": stats.get('FlatSpellBlockMod', 0),
            "Mana": stats.get('FlatMPPoolMod', 0),
            "AttackSpeed": stats.get('PercentAttackSpeedMod', 0),
            "MoveSpeed": stats.get('FlatMovementSpeedMod', 0),
            "CritChance": stats.get('FlatCritChanceMod', 0),
            
            
            "tag1": tags[0] if len(tags) > 0 else "None",
            "tag2": tags[1] if len(tags) > 1 else "None",
            "tag3": tags[2] if len(tags) > 2 else "None",
            
            #Extraction Regex ( les LARMES)
            "AbilityHaste": extract_stat_from_text(r'(\d+)\s*ability haste', clean_desc),
            "Lethality": extract_stat_from_text(r'(\d+)\s*lethality', clean_desc),
            "MagicPen": extract_stat_from_text(r'(\d+)\s*magic penetration', clean_desc),
            "Omnivamp": extract_stat_from_text(r'(\d+)%\s*omnivamp', clean_desc),
            "LifeSteal": extract_stat_from_text(r'(\d+)%\s*life steal', clean_desc),
            
            "ImageURL": f"https://ddragon.leagueoflegends.com/cdn/{version}/img/item/{details['image']['full']}"
        }

        items_list.append(row)

    df = pl.DataFrame(items_list)
    return df

#3 tourne stp
if __name__ == "__main__":
    df_objets = get_items_complete()
    
    if df_objets is not None and not df_objets.is_empty():
        print(f" Succès ! {len(df_objets)} objets récupérés.")
        print(df_objets.select(["Name", "Price", "AD", "AbilityHaste", "Lethality"]).head(10))
        
        # Sauvegarde
        # df_objets.write_csv("objets_complets_regression.csv", separator=";")
    else:
        print("Toujours vide... bizarre l'ambiance.")

Patch : 16.1.1 - Téléchargement via API Officielle...
Analyse de 694 objets...
 Succès ! 250 objets récupérés.
shape: (10, 5)
┌───────────────────┬───────┬─────┬──────────────┬───────────┐
│ Name              ┆ Price ┆ AD  ┆ AbilityHaste ┆ Lethality │
│ ---               ┆ ---   ┆ --- ┆ ---          ┆ ---       │
│ str               ┆ i64   ┆ i64 ┆ f64          ┆ f64       │
╞═══════════════════╪═══════╪═════╪══════════════╪═══════════╡
│ Boots             ┆ 300   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Faerie Charm      ┆ 200   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Rejuvenation Bead ┆ 300   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Giant's Belt      ┆ 900   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Cloak of Agility  ┆ 600   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Blasting Wand     ┆ 850   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Sapphire Crystal  ┆ 300   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Ruby Crystal      ┆ 400   ┆ 0   ┆ 0.0          ┆ 0.0       │
│ Cloth Armor       ┆ 300   ┆ 0   ┆ 0.0          ┆ 0.0 

sauvegarde

In [ ]:
#df_objets.write_csv("présentation_objets_complets.csv", separator=";")

modèles ?

In [ ]:
#1 On chope un df avec les colones utiles
colonnes_a_exclure = ["Name", "Price", "Sell_Price", "ImageURL", "ID","Sell_Price","tag1","tag2","tag3"]
X = df_objets.drop([c for c in colonnes_a_exclure if c in df_objets.columns])

y = df_objets.select("Price")

#2 Vérification
print("Colonnes utilisées pour la prédiction (X) :")
print(X.columns)

#3 Séparation train/test
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

Colonnes utilisées pour la prédiction (X) :
['AD', 'AP', 'HP', 'Armor', 'MR', 'Mana', 'AttackSpeed', 'MoveSpeed', 'CritChance', 'AbilityHaste', 'Lethality', 'MagicPen', 'Omnivamp', 'LifeSteal']


Recherche brute du meilleur modèle

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Liste des modèles 
modeles = [
    ("Dummy (Base)", DummyRegressor()), #Le modèle de base
    ("Linear", LinearRegression()),
    ("Lasso", Lasso(alpha=1.0)),
    ("Ridge", Ridge(alpha=1.0)),
    ("ElasticNet", ElasticNet(alpha=1.0, l1_ratio=0.5)),
    ("SVR", SVR(C=1.0, epsilon=0.1)),
    ("KNN", KNeighborsRegressor(n_neighbors=5)),
    ("GaussianProcess", GaussianProcessRegressor()),
    ("DecisionTree", DecisionTreeRegressor()),
    ("RandomForest", RandomForestRegressor()),
    ("GradientBoost", GradientBoostingRegressor()),
    ("NeuralNet", MLPRegressor(hidden_layer_sizes=(30,), max_iter=10000))
]

resultats = {}

print("Entraînement en cours...")

for nom, modele in modeles:
    pipe = make_pipeline(StandardScaler(), modele)
    
   
    scores = cross_val_score(pipe, X_tr, y_tr, cv=5, scoring='neg_mean_absolute_error')
    resultats[nom] = -scores 

# Création du DataFrame Polars
df_res = pl.DataFrame(resultats)

# Affichage des moyennes (classées par la plus petite erreur)
print("\n--- erreur moyenne en gold ---")
moyennes = df_res.mean().transpose(include_header=True, header_name="Modele", column_names=["Erreur_Moyenne"])
print(moyennes.sort("Erreur_Moyenne"))

Entraînement en cours...

--- erreur moyenne en gold ---
shape: (12, 2)
┌─────────────────┬────────────────┐
│ Modele          ┆ Erreur_Moyenne │
│ ---             ┆ ---            │
│ str             ┆ f64            │
╞═════════════════╪════════════════╡
│ RandomForest    ┆ 301.41527      │
│ DecisionTree    ┆ 310.615532     │
│ GradientBoost   ┆ 344.217077     │
│ Ridge           ┆ 346.397198     │
│ Linear          ┆ 346.497967     │
│ …               ┆ …              │
│ KNN             ┆ 418.131977     │
│ ElasticNet      ┆ 511.023323     │
│ SVR             ┆ 887.892069     │
│ Dummy (Base)    ┆ 958.3378       │
│ GaussianProcess ┆ 1804.267712    │
└─────────────────┴────────────────┘


Optimisation

In [ ]:
#1. On définit le modèle 
rf = RandomForestRegressor(random_state=42)

#2. On définit la "Grille" des hyperparamètres
param_grid = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [None, 10, 20],      
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4]     
}

#3. On lance la recherche (GridSearch)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1, 
    verbose=1
)

print("Optimisation de la Forêt en cours...")
grid_search.fit(X_tr, y_tr)

#4. Résultats
best_score = -grid_search.best_score_
best_params = grid_search.best_params_

print(f"\n MEILLEURE COMBINAISON TROUVÉE :")
print(f"Hyperparamètres : {best_params}")
print(f"Erreur Moyenne Optimisée : {best_score:.2f} Gold")

#5. Récupérer le modèle optimisé
mon_super_modele = grid_search.best_estimator_

Optimisation de la Forêt en cours...
Fitting 5 folds for each of 81 candidates, totalling 405 fits

 MEILLEURE COMBINAISON TROUVÉE :
Hyperparamètres : {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Erreur Moyenne Optimisée : 298.56 Gold


Validation ?


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

#1. Faire les prédictions
predictions = mon_super_modele.predict(X_te)

#2. Calculer les scores
mae = mean_absolute_error(y_te, predictions)
r2 = r2_score(y_te, predictions)

print(f"--- RÉSULTATS FINAUX SUR LE TEST SET ---")
print(f" Erreur Moyenne (MAE) : {mae:.2f} Gold")
print(f" Précision (R²)       : {r2:.2f} ")

#3. Créer un tableau comparatif
df_verif = pl.DataFrame({
    "Prix_Reel": y_te.to_numpy().flatten(),
    "Prix_Predit": predictions.flatten()
})


df_verif = df_verif.with_columns(
    (pl.col("Prix_Predit") - pl.col("Prix_Reel")).alias("Ecart")
)

#4. Afficher les plus grosses erreurs
print("\n--- TOP 5 DES PLUS GROSSES ERREURS ---")
# On ajoute .round(2) pour que ce soit plus grrr
print(df_verif.with_columns(pl.col("Ecart").abs().alias("Abs_Ecart"))
              .sort("Abs_Ecart", descending=True)
              .head(5))

--- RÉSULTATS FINAUX SUR LE TEST SET ---
 Erreur Moyenne (MAE) : 387.47 Gold
 Précision (R²)       : 0.66 (1.0 = Parfait, 0.0 = Nul)

--- TOP 5 DES PLUS GROSSES ERREURS ---
shape: (5, 4)
┌───────────┬─────────────┬──────────────┬─────────────┐
│ Prix_Reel ┆ Prix_Predit ┆ Ecart        ┆ Abs_Ecart   │
│ ---       ┆ ---         ┆ ---          ┆ ---         │
│ i64       ┆ f64         ┆ f64          ┆ f64         │
╞═══════════╪═════════════╪══════════════╪═════════════╡
│ 2500      ┆ 346.332866  ┆ -2153.667134 ┆ 2153.667134 │
│ 950       ┆ 2996.91373  ┆ 2046.91373   ┆ 2046.91373  │
│ 900       ┆ 2877.040476 ┆ 1977.040476  ┆ 1977.040476 │
│ 1300      ┆ 3029.976444 ┆ 1729.976444  ┆ 1729.976444 │
│ 1150      ┆ 2180.812698 ┆ 1030.812698  ┆ 1030.812698 │
└───────────┴─────────────┴──────────────┴─────────────┘


Nouvelle recherche

In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split



reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)


models, predictions = reg.fit(
    X_tr.to_pandas(), 
    X_te.to_pandas(), 
    y_tr.to_pandas(), 
    y_te.to_pandas()
)

print(models)

  0%|          | 0/42 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000530 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 65
[LightGBM] [Info] Number of data points in the train set: 187, number of used features: 6
[LightGBM] [Info] Start training from score 1999.839572
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd

#1 Création du Pipeline

pipe_huber = make_pipeline(StandardScaler(), HuberRegressor(epsilon=1.35, max_iter=2000))

#2. Entraînement
print("Entraînement du HuberRegressor...")
# On utilise .values.ravel() pour transformer y en un simple tableau 1D (évite les warnings)
pipe_huber.fit(X_tr.to_pandas(), y_tr.to_pandas().values.ravel())

#3. Prédictions
y_pred_train = pipe_huber.predict(X_tr.to_pandas())
y_pred_test = pipe_huber.predict(X_te.to_pandas())

#4. Calcul des scores
r2_train = r2_score(y_tr.to_pandas(), y_pred_train)
r2_test = r2_score(y_te.to_pandas(), y_pred_test)
mae_test = mean_absolute_error(y_te.to_pandas(), y_pred_test)

print("\n--- RÉSULTATS HUBER REGRESSOR ---")
print(f"R² Train (Apprentissage) : {r2_train:.3f}")
print(f"R² Test (Généralisation) : {r2_test:.3f}")
print(f"Erreur Moyenne (MAE)     : {mae_test:.1f} Gold")

#5. Extraction de la Gold Value
huber_model = pipe_huber.steps[1][1] 


#terminado
print(f"Huber R² Test : {r2_test:.3f}")


if r2_test > 0.60:
    print("L'économie de LoL semble vachement bien linéaire !")
else:
    print("Eh vasy")

🏋️ Entraînement du HuberRegressor...

--- 📊 RÉSULTATS HUBER REGRESSOR ---
R² Train (Apprentissage) : 0.839
R² Test (Généralisation) : 0.769
Erreur Moyenne (MAE)     : 359.9 Gold

--- ⚖️ COMPARAISON VS RANDOM FOREST ---
Huber R² Test : 0.769
Forest R² Test: (Votre Score Précédent, probablement ~0.55 - 0.60)
✅ LE HUBER GAGNE : L'économie de LoL est bien linéaire !


Enregistrement du modèle final + utilisation

In [ ]:
import joblib
from sklearn.linear_model import HuberRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

#1 : On prend la liste des colonnes
cols_features = [
    "AD", "AP", "HP", "Armor", "MR", "Mana", 
    "AttackSpeed", "MoveSpeed", "CritChance", 
    "AbilityHaste", "Lethality", "MagicPen", "Omnivamp", "LifeSteal"
]

#2 : On prépare les données d'entraînement (converssion aussi en vrai)
X = df_objets.select(cols_features).to_pandas()
y = df_objets.select("Price").to_pandas().values.ravel() # .ravel() aplatit le tableau

#3 : On crée et entraîne le Pipeline Final 
modele_final = make_pipeline(
    StandardScaler(), 
    HuberRegressor(epsilon=1.35, max_iter=2000)
)

print("Entraînement final en cours...")
modele_final.fit(X, y)
print("Modèle entraîné !")

#4 : On sauvegarde les paramètres
package = {
    "pipeline": modele_final, 
    "features": cols_features, 
    "description": "Modèle HuberRegressor pour prédiction prix LoL"
}

#5 : Enregistrement sur le disque dur ( god bless)
filename = "lol_price_model.pkl"
joblib.dump(package, filename)

print(f"Le fichier '{filename}' est prêt.")


⏳ Entraînement final en cours...
✅ Modèle entraîné !
🎉 Succès ! Le fichier 'lol_price_model.pkl' est prêt.
